In [29]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [30]:
START_DATE = '2023-11-25'
END_DATE = '2024-11-23'
INVESTMENT = 1000000


In [31]:
# Function to calculate the purchasing fee depending on the number of shares
def calculate_purchase_fee(shares):
    return max(3.95, 0.01*shares)

In [32]:

def validity(tickers):
    final_list = []
    for ticker in tickers:
        #only append the stock to the final stock list if it is valid
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_trading_days(ticker)):
                final_list.append(ticker)
    return final_list

def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock - AI
            if 'Close' not in data.columns or data['Close'].isnull().all():
                #if there are no valid close prices, the stock is delisted
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

def check_trading_days(ticker):
    min_days=18
    #the stock should have at least 18 trading days (we can check this by checking the last month)
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period="1mo") #fetches the stock info from the last month
        #AI
        trading_days = data.shape[0]
        return trading_days >= min_days  # Must have at least `min_days` trading days
    except Exception as e:
        return False  # If error, assume it’s invalid

def check_currency(ticker):
    """Check if the stock is in USD or CAD currency."""
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # If error, assume it’s invalid

all_data = pd.read_csv('Tickers_Example.csv', header=None)

all_data = pd.DataFrame(validity(all_data[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)

$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [33]:
# Create a list of all the stock betas (retrieved from yfinance) of stock tickers in a list
def get_betas(tickers):
    betas = []
    
    for ticker in tickers:
        betas.append(yf.Ticker(ticker).info.get('beta'))

    return betas

In [34]:
def get_sector(tickers):
    sectors = []

    for ticker in tickers:
        sectors.append(yf.Ticker(ticker).info.get('sector'))

    return sectors

In [ ]:
# Add a column containing all the stock betas to [all_data]
all_data['Beta'] = get_betas(all_data['Ticker'])
all_data['Sector'] = get_sector(all_data['Ticker'])

stocks = [x for _, x in all_data.groupby('Sector')]
stocks = list(map(filter_betas, stocks))
stocks

In [48]:
# Choose the max(24, data.length) stocks that match the market the closest
def filter_betas(data):
    # Remove any negative betas, as long as data ends with at least 12 characters
    data.sort_values(by='Beta', axis=0, inplace=True, kind='quicksort')
    data.drop(index=data[(data['Beta'] <= 0) & (len(data) > 12)].index, inplace=True)
    data.reset_index(drop=True, inplace=True)

    data['temp'] = abs(1-data['Beta'])
    data.sort_values(by='temp', axis=0, inplace=True, kind='quicksort')    # Sort data by temp
    data.drop(columns='temp', axis=1, inplace=True)    # Remove temp

    data.index = range(0, len(data))    # Reassign the index
    
    return data[:24] if len(data) > 24 else data    # Return first 24 elements of data if it is greater than 24, other wise return data

In [49]:
filter_betas(all_data)
all_data.index = all_data.index + 1

In [38]:
tsx60_info = yf.Ticker('XIU.TO')
tsx60 = tsx60_info.history(start=START_DATE, end=END_DATE, interval='1d')

sp500_info = yf.Ticker('^GSPC')
sp500 = sp500_info.history(start=START_DATE, end=END_DATE, interval='1d')
sp500 = sp500.reindex(tsx60.index, method='nearest')

market_returns = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
market_returns.index = market_returns.index.strftime('%Y-%m-%d')
market_returns = market_returns.dropna()
market_returns_mean = market_returns.mean()

market_returns_df = pd.DataFrame(market_returns)

In [39]:
# all_data
ticker_returns = pd.DataFrame(columns=all_data['Ticker'])
for ticker in all_data['Ticker']:
        data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close']
        ticker_returns[ticker] = data.pct_change().dropna()  # Calculate daily returns

ticker_returns.index = ticker_returns.index.strftime('%Y-%m-%d')

common_dates = ticker_returns.index.intersection(market_returns_df.index)
ticker_returns = ticker_returns.loc[common_dates]
market_returns_df = market_returns_df.loc[common_dates]

ticker_returns = ticker_returns.dropna()
market_returns_df = market_returns_df.dropna()

In [40]:
portfolio_data = []


def portfolio_generator(tickers):
    global portfolio_data
    min_result = float('inf')
    num_stocks = 0 
    final_weights = 0

    for i in range(12, 25):  
        selected_tickers = tickers[:i]  
        # print(f"Iteration {i}, Selected Tickers: {tickers[:i]}")


        def objective(weights):
            return np.std(np.dot(ticker_returns[selected_tickers].dropna().values, weights) - market_returns_df.dropna().values)
        
        # Initial weights (equal distribution) - AI
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # Constraints: Weights must sum to 1 - AI
        constraints = {'type': 'eq', 'fun': lambda w: sum(w) - 1}
        
        # Bounds: Weights between 0 and 1- AI
        bounds = [(0, 1)] * len(selected_tickers)
        
        result = minimize(objective, initial_weights, constraints=[constraints], bounds=bounds)
        # print(result.fun)

        
        if result.success:
            if result.fun < min_result:
                min_result = result.fun
                num_stocks = i
                final_weights = result.x
        
        portfolio_data.append({
            'tickers': ', '.join(selected_tickers), #AI
            'weights': final_weights,
            'STD Between Stock and Market Index': min_result,
            'num_stocks': num_stocks
        })
        
    return [pd.DataFrame(portfolio_data), min_result]
    
# all_data['Ticker']
portfolio_df = portfolio_generator(all_data['Ticker'])[0]
min_std = portfolio_generator(all_data['Ticker'])[1]

portfolio_df

,tickers,weights,STD Between Stock and Market Index,num_stocks
0,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.08133928579604588, 0.08124022107353729, 0.0...",0.008844,12
1,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.07498226834562435, 0.07725642833113375, 0.0...",0.008825,13
2,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.06202503855776728, 0.060859237238410344, 0....",0.008684,14
3,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.05541081472816768, 0.05370390969577994, 0.0...",0.008660,15
4,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.041433176597750046, 0.05654953062352324, 0....",0.008366,16
5,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.03570393053124705, 0.05085927145466591, 0.0...",0.008181,17
6,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.03112232369969745, 0.05805910360930995, 0.0...",0.008156,18
7,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.03255919871012827, 0.05791125536014425, 0.0...",0.008155,19
8,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.03255919871012827, 0.05791125536014425, 0.0...",0.008155,19
9,"MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV...","[0.0233945918134597, 0.06737034328892419, 0.06...",0.008144,21


In [47]:
portfolio_data

[{'tickers': 'MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV, PFE, KO',
  'weights': array([0.08133929, 0.08124022, 0.09307633, 0.07176578, 0.01893892,
         0.21183568, 0.09367637, 0.09260565, 0.07860807, 0.03614927,
         0.02585102, 0.1149134 ]),
  'STD Between Stock and Market Index': 0.008843961063670771,
  'num_stocks': 12},
 {'tickers': 'MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV, PFE, KO, MO',
  'weights': array([0.07498227, 0.07725643, 0.08711086, 0.07662765, 0.00878543,
         0.20752132, 0.08933693, 0.07452849, 0.07512342, 0.02935252,
         0.02414613, 0.10739715, 0.06783142]),
  'STD Between Stock and Market Index': 0.008825214996265452,
  'num_stocks': 13},
 {'tickers': 'MRK, PG, CL, LLY, BMY, LMT, PEP, PM, UNH, ABBV, PFE, KO, MO, T.TO',
  'weights': array([0.06202504, 0.06085924, 0.07658003, 0.07199021, 0.00332302,
         0.17904998, 0.07818818, 0.05665161, 0.06418034, 0.02906361,
         0.01742162, 0.09024863, 0.05665896, 0.15375954]),
  'STD Between S

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.